# Projekt: od surowych danych do prostego modelu ML 

**Autor:** Marcel Moraczewski
**Przedmiot:** python

Celem projektu jest pełna analiza danych pracowniczych – od EDA, przez preprocessing, aż do budowy i oceny prostego modelu ML.

## 1. Importy i wczytanie danych

## Opis problemu i hipoteza

**Problem:** przewidywanie wynagrodzenia pracownika (`Salary`) na podstawie cech demograficznych i zawodowych.

**Hipoteza:** Wynagrodzenie rośnie wraz z doświadczeniem, a największy wpływ mają lata doświadczenia, stanowisko oraz dział.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

df = pd.read_csv('Employers_data.csv')
df.head()

## 2. Pierwsze spojrzenie na dane

In [ ]:
df.shape
df.info()
df.describe(include='all')
df.isna().sum()

### Wnioski wstępne
- Zbiór zawiera 10 000 obserwacji i 10 kolumn.
- Brak brakujących wartości.
- Dane zawierają zarówno cechy numeryczne, jak i kategoryczne.
- Naturalnym targetem jest kolumna `Salary`.

## 3. EDA – analiza jednowymiarowa

In [ ]:
numeric_cols = df.select_dtypes(include=['int64','float64']).columns

for col in numeric_cols:
    plt.figure(figsize=(6,4))
    df[col].hist(bins=30)
    plt.title(f'Histogram: {col}')
    plt.show()

### Boxploty – wykrywanie obserwacji odstających

In [ ]:
for col in numeric_cols:
    plt.figure(figsize=(6,4))
    df.boxplot(column=col)
    plt.title(f'Boxplot: {col}')
    plt.show()

### Wnioski z EDA jednowymiarowego
- Wynagrodzenie ma rozkład prawostronnie skośny.
- Widoczne są obserwacje odstające w `Salary`.
- Doświadczenie jest silnie zróżnicowane w populacji.

## 4. EDA – analiza zależności

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(df['Experience_Years'], df['Salary'], alpha=0.4)
plt.xlabel('Experience Years')
plt.ylabel('Salary')
plt.title('Salary vs Experience')
plt.show()

### Średnie wynagrodzenie wg działu

In [ ]:
df.groupby('Department')['Salary'].mean().sort_values().plot(kind='barh', figsize=(6,4))
plt.title('Średnie wynagrodzenie wg działu')
plt.show()

## 5. Korelacje

In [ ]:
corr = df.corr(numeric_only=True)
plt.figure(figsize=(6,4))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Macierz korelacji')
plt.show()

### Wnioski z korelacji
- Najsilniejszą korelację z wynagrodzeniem ma liczba lat doświadczenia.
- Wiek jest pośrednio skorelowany z pensją.
- Employee_ID nie wnosi informacji predykcyjnej.

## 6. Preprocessing

In [ ]:
data = df.drop(['Employee_ID', 'Name'], axis=1)
data_encoded = pd.get_dummies(data, drop_first=True)

X = data_encoded.drop('Salary', axis=1)
y = data_encoded['Salary']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## 7. Model ML – regresja liniowa

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse, mae

## 8. Podsumowanie i wnioski końcowe
- Dane były wysokiej jakości i nie wymagały imputacji braków.
- Najważniejszą cechą okazało się doświadczenie zawodowe.
- Regresja liniowa sprawdziła się jako model bazowy.
- Dalsze kroki: test modeli nieliniowych, selekcja cech, walidacja krzyżowa.